In [ ]:
#%matplotlib inline
import scipy
from scipy import sparse
from scipy.sparse.linalg.dsolve import linsolve
import numpy as np

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import matplotlib.pyplot as plt

class Fdm:
    'grille differences finies'
    fdmCount = 0

    def __init__(self, Lx, Ly, Nx, Ny):
        self.Lx = Lx
        self.Ly = Ly
        self.Nx = Nx
        self.Ny = Ny 
        self.N  = Nx*Ny
        self.dx = Lx/(Nx-1);
        self.dy = Ly/(Ny-1);
        Fdm.fdmCount += 1
   
    def displayCount(self):
        print ("Instances {:d}".format(Fdm.fdmCount)) 
            
    def index(self, i, j):
        return self.Nx*j+i
    
    def __dirichlet(self):
        self.ld = self.index(np.arange(self.Nx), 0) 
        ld = self.index(np.arange(self.Nx), self.Ny-1)
        self.ld=np.hstack((self.ld, ld))
        ld = self.index(0, np.arange(self.Ny))
        self.ld=np.hstack((self.ld, ld))
        ld = self.index(self.Nx-1, np.arange(self.Ny))
        self.ld=np.hstack((self.ld, ld))                  
        self.ld=np.unique(self.ld)

    def __build_K(self):    
        row  = []
        col  = []
        data = []

        D=1/self.dx**2+1/self.dy**2;
        M=10*D
        
        for ix in range(self.Nx):
            for iy in range(self.Ny):
                n=self.index(ix, iy)  
                if n in self.ld: # test d'appartenance
                    continue
                else:
                    row.append(n)
                    col.append(n)
                    data.append(2*D)
                    
                    p=self.index(ix+1, iy)
                    row.append(n)
                    col.append(p)
                    data.append(-1/self.dx**2)

                    p=self.index(ix-1, iy)
                    row.append(n)
                    col.append(p)
                    data.append(-1/self.dx**2)
                    
                    p=self.index(ix, iy+1)
                    row.append(n)
                    col.append(p)
                    data.append(-1/self.dy**2)
                    
                    p=self.index(ix, iy-1)
                    row.append(n)
                    col.append(p)
                    data.append(-1/self.dy**2)
                    
        K=sparse.csr_matrix( (data,(row,col)), shape=(self.N, self.N) )
        return K

    def solve(self, n):
        self.__dirichlet();
        K = self.__build_K();
        UN= np.ones(self.N)
        P=sparse.diags(UN).tocsr()
        P=delete_rows(P, fdm.ld)
        
        Kp=P @ K @ P.T
        vals, vecs = sparse.linalg.eigsh(Kp, k=n, which='SM')
        vecs = P.T*vecs
        print(vecs.shape)
        
        fig = plt.figure(1)
        ax = fig.gca(projection='3d')
        X = np.linspace(0, self.Lx, num=self.Nx, endpoint=True)
        xlen = len(X)
        Y = np.linspace(0, self.Ly, num=self.Ny, endpoint=True)
        ylen = len(Y)
        X, Y = np.meshgrid(X, Y)
        print(X)
        print(vecs[:, n-1])
        Z = vecs[:, n-1].reshape(self.Nx, self.Ny)

        colortuple = ('y', 'b')
        colors = np.empty(X.shape, dtype=str)
        
        surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap='jet',
                       linewidth=0, antialiased=False)

        fig.colorbar(surf)
        plt.figure(2)
        CS = plt.contour(X, Y, Z, 11,
                 colors='k',  # negative contours will be dashed by default
                 )
        plt.clabel(CS, inline=1, fontsize=10)
        plt.title('Simplest default with labels')

        plt.show()

def delete_rows(mat, indices):
    valid = isinstance(mat, scipy.sparse.lil_matrix) or \
            isinstance(mat, scipy.sparse.csr_matrix)
    if not valid:
        raise ValueError("works only for LIL / CSR format -- use .tolil() / .tocsr() first")
    indices = list(indices)
    mask = np.ones(mat.shape[0], dtype=bool)
    mask[indices] = False
    return mat[mask]


fdm = Fdm(1, 1, 64, 64)
fdm.solve(3)


(4096, 3)
[[ 0.          0.01587302  0.03174603 ...,  0.96825397  0.98412698  1.        ]
 [ 0.          0.01587302  0.03174603 ...,  0.96825397  0.98412698  1.        ]
 [ 0.          0.01587302  0.03174603 ...,  0.96825397  0.98412698  1.        ]
 ..., 
 [ 0.          0.01587302  0.03174603 ...,  0.96825397  0.98412698  1.        ]
 [ 0.          0.01587302  0.03174603 ...,  0.96825397  0.98412698  1.        ]
 [ 0.          0.01587302  0.03174603 ...,  0.96825397  0.98412698  1.        ]]
[ 0.  0.  0. ...,  0.  0.  0.]
